In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from collections import Counter
import math
import time
import json

# Create necessary directories
os.makedirs('LogisticRegression', exist_ok=True)
os.makedirs('DecisionTree', exist_ok=True)
os.makedirs('KNN', exist_ok=True)

# Load the datasets
print("Loading datasets...")
train_data = pd.read_csv('preprocessed_data/train_data.csv')
validation_data = pd.read_csv('preprocessed_data/validation_data.csv')
test_data = pd.read_csv('preprocessed_data/test_data.csv')

# Separate features and target
X_train = train_data.drop('default_payment', axis=1).values
y_train = train_data['default_payment'].values
X_val = validation_data.drop('default_payment', axis=1).values
y_val = validation_data['default_payment'].values
X_test = test_data.drop('default_payment', axis=1).values
y_test = test_data['default_payment'].values

# Helper functions for evaluation
def calculate_metrics(y_true, y_pred, probabilities=None):
    """Calculate accuracy, precision, recall, F1-score, and AUC."""
    # True positives, false positives, true negatives, false negatives
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate metrics
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate AUC if probabilities are provided
    auc = None
    if probabilities is not None:
        # Sort by probability and calculate AUC
        sorted_indices = np.argsort(probabilities)
        sorted_y_true = y_true[sorted_indices]
        
        # Calculate AUC using trapezoidal rule
        positive_count = np.sum(y_true == 1)
        negative_count = np.sum(y_true == 0)
        
        if positive_count > 0 and negative_count > 0:
            fp_rates = []
            tp_rates = []
            thresholds = np.unique(probabilities)
            thresholds = np.append(thresholds, thresholds[-1] + 1)  # Add a threshold higher than the max
            thresholds = np.sort(thresholds)
            
            for threshold in thresholds:
                y_pred_temp = (probabilities >= threshold).astype(int)
                tp_temp = np.sum((y_true == 1) & (y_pred_temp == 1))
                fp_temp = np.sum((y_true == 0) & (y_pred_temp == 1))
                tp_rate = tp_temp / positive_count if positive_count > 0 else 0
                fp_rate = fp_temp / negative_count if negative_count > 0 else 0
                tp_rates.append(tp_rate)
                fp_rates.append(fp_rate)
            
            # Calculate AUC using trapezoidal rule
            auc = 0.0
            for i in range(len(fp_rates) - 1):
                auc += (fp_rates[i+1] - fp_rates[i]) * (tp_rates[i+1] + tp_rates[i]) / 2
            
            # Convert from increasing FPR to decreasing FPR for proper AUC calculation
            auc = 1.0 - auc
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc
    }

def plot_roc_curve(y_true, y_prob, classifier_name, output_dir):
    """Plot ROC curve and save to file"""
    positive_count = np.sum(y_true == 1)
    negative_count = np.sum(y_true == 0)
    
    thresholds = np.unique(y_prob)
    thresholds = np.append(thresholds, thresholds[-1] + 1)  # Add a threshold higher than max
    thresholds = np.sort(thresholds)
    
    fp_rates = []
    tp_rates = []
    
    for threshold in thresholds:
        y_pred_temp = (y_prob >= threshold).astype(int)
        tp_temp = np.sum((y_true == 1) & (y_pred_temp == 1))
        fp_temp = np.sum((y_true == 0) & (y_pred_temp == 1))
        tp_rate = tp_temp / positive_count if positive_count > 0 else 0
        fp_rate = fp_temp / negative_count if negative_count > 0 else 0
        tp_rates.append(tp_rate)
        fp_rates.append(fp_rate)
    
    # Sort for proper plotting
    points = sorted(zip(fp_rates, tp_rates))
    fp_rates = [p[0] for p in points]
    tp_rates = [p[1] for p in points]
    
    # Calculate AUC using trapezoidal rule
    auc = 0
    for i in range(len(fp_rates) - 1):
        auc += (fp_rates[i+1] - fp_rates[i]) * (tp_rates[i+1] + tp_rates[i]) / 2
    
    # Plot ROC curve
    plt.figure(figsize=(10, 8))
    plt.plot(fp_rates, tp_rates, color='blue', lw=2, label=f'ROC curve (AUC = {auc:.3f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {classifier_name}')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.savefig(f"{output_dir}/roc_curve.png")
    plt.close()

def confusion_matrix_plot(y_true, y_pred, classifier_name, output_dir):
    """Plot confusion matrix and save to file"""
    # Calculate confusion matrix
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    cm = np.array([[tn, fp], [fn, tp]])
    
    # Plot
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {classifier_name}')
    plt.colorbar()
    
    # Add labels
    classes = ['No Default (0)', 'Default (1)']
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    # Add text annotations
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f"{output_dir}/confusion_matrix.png")
    plt.close()

def save_results(metrics, training_time, classifier_name, output_dir):
    """Save metrics and training time to a JSON file"""
    results = {
        'metrics': metrics,
        'training_time_seconds': training_time
    }
    
    with open(f"{output_dir}/results.json", 'w') as f:
        json.dump(results, f, indent=4)
    
    # Also save as text for easy reading
    with open(f"{output_dir}/results.txt", 'w') as f:
        f.write(f"Results for {classifier_name}\n")
        f.write(f"Training time: {training_time:.4f} seconds\n\n")
        f.write("Test Set Metrics:\n")
        for metric, value in metrics.items():
            if value is not None:
                f.write(f"{metric}: {value:.4f}\n")
            else:
                f.write(f"{metric}: N/A\n")

# ========== 1. Logistic Regression from Scratch ==========
class LogisticRegression:
    def __init__(self, learning_rate=0.01, max_iterations=1000, tol=1e-4):
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.tol = tol
        self.weights = None
        self.bias = None
        self.losses = []
    
    def sigmoid(self, z):
        """Sigmoid activation function"""
        # Clip to avoid overflow
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def initialize_parameters(self, n_features):
        """Initialize weights and bias"""
        self.weights = np.zeros(n_features)
        self.bias = 0
    
    def compute_cost(self, X, y):
        """Compute the binary cross-entropy loss"""
        m = X.shape[0]
        z = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(z)
        
        # Avoid log(0) by adding a small epsilon
        epsilon = 1e-15
        predictions = np.clip(predictions, epsilon, 1 - epsilon)
        
        # Binary cross-entropy loss
        loss = -1/m * np.sum(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
        return loss
    
    def compute_gradients(self, X, y):
        """Compute gradients for weights and bias"""
        m = X.shape[0]
        z = np.dot(X, self.weights) + self.bias
        predictions = self.sigmoid(z)
        
        # Gradients
        dw = 1/m * np.dot(X.T, (predictions - y))
        db = 1/m * np.sum(predictions - y)
        
        return dw, db
    
    def fit(self, X, y, verbose=True):
        """Train the logistic regression model using gradient descent"""
        # Initialize parameters
        self.initialize_parameters(X.shape[1])
        
        # Gradient descent
        prev_loss = float('inf')
        for i in range(self.max_iterations):
            # Compute gradients
            dw, db = self.compute_gradients(X, y)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
            # Compute loss
            if i % 100 == 0 or i == self.max_iterations - 1:
                loss = self.compute_cost(X, y)
                self.losses.append(loss)
                
                if verbose and (i % 100 == 0):
                    print(f"Iteration {i}, Loss: {loss:.4f}")
                
                # Check for convergence
                if abs(prev_loss - loss) < self.tol:
                    if verbose:
                        print(f"Converged at iteration {i} with loss {loss:.4f}")
                    break
                
                prev_loss = loss
    
    def predict_proba(self, X):
        """Predict probability of the positive class"""
        z = np.dot(X, self.weights) + self.bias
        return self.sigmoid(z)
    
    def predict(self, X, threshold=0.5):
        """Predict class labels"""
        probabilities = self.predict_proba(X)
        return (probabilities >= threshold).astype(int)

# Train and evaluate Logistic Regression
print("\n===== Implementing Logistic Regression from Scratch =====")
log_reg = LogisticRegression(learning_rate=0.01, max_iterations=1000)

start_time = time.time()
print("Training logistic regression...")
log_reg.fit(X_train, y_train)
lr_training_time = time.time() - start_time
print(f"Training completed in {lr_training_time:.4f} seconds")

# Evaluate on test set
print("Evaluating on test set...")
y_prob_lr = log_reg.predict_proba(X_test)
y_pred_lr = log_reg.predict(X_test)
lr_metrics = calculate_metrics(y_test, y_pred_lr, y_prob_lr)

# Plot learning curve
plt.figure(figsize=(10, 6))
plt.plot(range(0, len(log_reg.losses) * 100, 100), log_reg.losses)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Logistic Regression Learning Curve')
plt.grid(True)
plt.savefig('LogisticRegression/learning_curve.png')
plt.close()

# Plot ROC curve and confusion matrix
plot_roc_curve(y_test, y_prob_lr, "Logistic Regression", "LogisticRegression")
confusion_matrix_plot(y_test, y_pred_lr, "Logistic Regression", "LogisticRegression")

# Save model parameters
np.savetxt('LogisticRegression/weights.txt', log_reg.weights)
with open('LogisticRegression/bias.txt', 'w') as f:
    f.write(str(log_reg.bias))

# Save results
save_results(lr_metrics, lr_training_time, "Logistic Regression", "LogisticRegression")
print("Logistic Regression results saved in LogisticRegression directory")

# ========== 2. Decision Tree Classifier from Scratch ==========
class DecisionTreeNode:
    def __init__(self, feature_idx=None, threshold=None, left=None, right=None, value=None):
        # For decision nodes
        self.feature_idx = feature_idx  # Index of the feature to split on
        self.threshold = threshold      # Threshold value for the split
        self.left = left                # Left subtree (samples where feature <= threshold)
        self.right = right              # Right subtree (samples where feature > threshold)
        
        # For leaf nodes
        self.value = value              # Class prediction for a leaf node

class DecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None
        self.feature_importances_ = None
    
    def fit(self, X, y):
        """Build the decision tree"""
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)
        
        # Calculate feature importances
        self.feature_importances_ = np.zeros(self.n_features)
        self._calculate_feature_importances(self.tree, self.n_features)
        
        # Normalize feature importances
        if np.sum(self.feature_importances_) > 0:
            self.feature_importances_ /= np.sum(self.feature_importances_)
    
    def _calculate_feature_importances(self, node, n_features, depth=0):
        """Recursively calculate feature importances"""
        if node.feature_idx is not None:  # Decision node
            # Importance is proportional to the depth
            importance = 1.0 / (depth + 1.0)
            self.feature_importances_[node.feature_idx] += importance
            
            # Recurse down the tree
            self._calculate_feature_importances(node.left, n_features, depth + 1)
            self._calculate_feature_importances(node.right, n_features, depth + 1)
    
    def _grow_tree(self, X, y, depth=0):
        """Recursively grow the decision tree"""
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        
        # Check stopping criteria
        if (self.max_depth is not None and depth >= self.max_depth or
            n_samples < self.min_samples_split or
            n_classes == 1):
            leaf_value = self._most_common_label(y)
            return DecisionTreeNode(value=leaf_value)
        
        # Find the best split
        best_feature, best_threshold = self._best_split(X, y)
        
        # If no improvement, create a leaf node
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return DecisionTreeNode(value=leaf_value)
        
        # Split the data
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices
        
        # Check if the split is valid (both sides have minimum samples)
        if np.sum(left_indices) < self.min_samples_leaf or np.sum(right_indices) < self.min_samples_leaf:
            leaf_value = self._most_common_label(y)
            return DecisionTreeNode(value=leaf_value)
        
        # Grow the left and right subtrees
        left_subtree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        
        return DecisionTreeNode(
            feature_idx=best_feature,
            threshold=best_threshold,
            left=left_subtree,
            right=right_subtree
        )
    
    def _best_split(self, X, y):
        """Find the best feature and threshold for splitting"""
        m, n = X.shape
        if m <= 1:
            return None, None
        
        # Parent set entropy
        parent_entropy = self._entropy(y)
        
        # If parent entropy is 0, no need to split
        if parent_entropy == 0:
            return None, None
        
        # Initialize variables
        best_info_gain = -float('inf')
        best_feature = None
        best_threshold = None
        
        # Try each feature
        for feature_idx in range(n):
            feature_values = X[:, feature_idx]
            thresholds = np.unique(feature_values)
            
            # Try each threshold
            for threshold in thresholds:
                # Split the data
                left_indices = feature_values <= threshold
                right_indices = ~left_indices
                
                # Skip if split doesn't meet minimum samples
                if np.sum(left_indices) < self.min_samples_leaf or np.sum(right_indices) < self.min_samples_leaf:
                    continue
                
                # Calculate information gain
                left_entropy = self._entropy(y[left_indices])
                right_entropy = self._entropy(y[right_indices])
                
                # Weighted sum of entropies
                n_left, n_right = np.sum(left_indices), np.sum(right_indices)
                weighted_entropy = (n_left * left_entropy + n_right * right_entropy) / m
                
                # Information gain
                info_gain = parent_entropy - weighted_entropy
                
                # Update best split if this is better
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature = feature_idx
                    best_threshold = threshold
        
        return best_feature, best_threshold
    
    def _entropy(self, y):
        """Calculate entropy of a set"""
        if len(y) == 0:
            return 0
        
        # Count occurrences of each class
        counts = np.bincount(y.astype(int))
        probabilities = counts / len(y)
        
        # Remove zero probabilities
        probabilities = probabilities[probabilities > 0]
        
        # Calculate entropy
        return -np.sum(probabilities * np.log2(probabilities))
    
    def _most_common_label(self, y):
        """Return the most common label in a set"""
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    def predict(self, X):
        """Predict class labels for samples in X"""
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def predict_proba(self, X):
        """Predict class probabilities for samples in X"""
        # This is a simplification - we're using a basic approach where probability
        # is 1 for the predicted class and 0 for others
        # In a more sophisticated implementation, we would use the class distribution in the leaf
        y_pred = self.predict(X)
        return y_pred  # Binary classification (0 or 1)
    
    def _traverse_tree(self, x, node):
        """Traverse the tree to make a prediction for a single sample"""
        # If leaf node, return the predicted value
        if node.value is not None:
            return node.value
        
        # Traverse left or right based on the feature value
        if x[node.feature_idx] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    def print_tree(self, node=None, depth=0):
        """Print the decision tree structure (for debugging)"""
        if node is None:
            node = self.tree
        
        # Indent based on depth
        indent = "  " * depth
        
        # Leaf node
        if node.value is not None:
            print(f"{indent}Predict: {node.value}")
        # Decision node
        else:
            print(f"{indent}Feature {node.feature_idx} <= {node.threshold}")
            print(f"{indent}Left subtree:")
            self.print_tree(node.left, depth + 1)
            print(f"{indent}Right subtree:")
            self.print_tree(node.right, depth + 1)

# Train and evaluate Decision Tree
print("\n===== Implementing Decision Tree Classifier from Scratch =====")
dt = DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5)

start_time = time.time()
print("Training decision tree...")
dt.fit(X_train, y_train)
dt_training_time = time.time() - start_time
print(f"Training completed in {dt_training_time:.4f} seconds")

# Evaluate on test set
print("Evaluating on test set...")
y_pred_dt = dt.predict(X_test)
y_prob_dt = dt.predict(X_test)  # Simple prediction as probability
dt_metrics = calculate_metrics(y_test, y_pred_dt, y_prob_dt)

# Plot feature importances
plt.figure(figsize=(12, 6))
indices = np.argsort(dt.feature_importances_)[-10:]  # Top 10 features
plt.barh(range(len(indices)), dt.feature_importances_[indices], align='center')
feature_names = train_data.drop('default_payment', axis=1).columns
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.title('Top 10 Feature Importances in Decision Tree')
plt.savefig('DecisionTree/feature_importances.png')
plt.close()

# Plot confusion matrix
confusion_matrix_plot(y_test, y_pred_dt, "Decision Tree", "DecisionTree")

# Save results
save_results(dt_metrics, dt_training_time, "Decision Tree", "DecisionTree")
print("Decision Tree results saved in DecisionTree directory")

# ========== 3. K-Nearest Neighbors Classifier from Scratch ==========
class KNNClassifier:
    def __init__(self, k=5):
        self.k = k
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """Store the training data"""
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        """Predict class labels for samples in X"""
        return np.array([self._predict_single(x) for x in X])
    
    def predict_proba(self, X):
        """Predict class probabilities for samples in X"""
        # For binary classification, return the proportion of positive class neighbors
        probabilities = np.array([self._predict_proba_single(x) for x in X])
        return probabilities
    
    def _predict_single(self, x):
        """Predict the class for a single sample"""
        # Calculate distances
        distances = np.sqrt(np.sum((self.X_train - x)**2, axis=1))
        
        # Find indices of k nearest neighbors
        nearest_indices = np.argsort(distances)[:self.k]
        
        # Get labels of k nearest neighbors
        nearest_labels = self.y_train[nearest_indices]
        
        # Majority vote
        most_common = Counter(nearest_labels).most_common(1)[0][0]
        return most_common
    
    def _predict_proba_single(self, x):
        """Predict the probability for a single sample"""
        # Calculate distances
        distances = np.sqrt(np.sum((self.X_train - x)**2, axis=1))
        
        # Find indices of k nearest neighbors
        nearest_indices = np.argsort(distances)[:self.k]
        
        # Get labels of k nearest neighbors
        nearest_labels = self.y_train[nearest_indices]
        
        # Calculate proportion of positive class (class 1)
        return np.mean(nearest_labels)

# Train and evaluate KNN
print("\n===== Implementing K-Nearest Neighbors Classifier from Scratch =====")
knn = KNNClassifier(k=5)

start_time = time.time()
print("Training KNN...")
knn.fit(X_train, y_train)
knn_training_time = time.time() - start_time
print(f"Training completed in {knn_training_time:.4f} seconds")

# Evaluate on test set
print("Evaluating on test set...")
y_pred_knn = knn.predict(X_test)
y_prob_knn = knn.predict_proba(X_test)
knn_metrics = calculate_metrics(y_test, y_pred_knn, y_prob_knn)

# Plot ROC curve and confusion matrix
plot_roc_curve(y_test, y_prob_knn, "K-Nearest Neighbors", "KNN")
confusion_matrix_plot(y_test, y_pred_knn, "K-Nearest Neighbors", "KNN")

# Save results
save_results(knn_metrics, knn_training_time, "K-Nearest Neighbors", "KNN")
print("KNN results saved in KNN directory")

# ========== Compare all models ==========
print("\n===== Comparing All Models =====")
models = ["Logistic Regression", "Decision Tree", "KNN"]
metrics_list = [lr_metrics, dt_metrics, knn_metrics]
training_times = [lr_training_time, dt_training_time, knn_training_time]

# Create comparison table
comparison = pd.DataFrame({
    'Model': models,
    'Accuracy': [m['accuracy'] for m in metrics_list],
    'Precision': [m['precision'] for m in metrics_list],
    'Recall': [m['recall'] for m in metrics_list],
    'F1 Score': [m['f1_score'] for m in metrics_list],
    'AUC': [m['auc'] if m['auc'] is not None else float('nan') for m in metrics_list],
    'Training Time (s)': training_times
})

print("\nModel Comparison:")
print(comparison)

# Save comparison results
comparison.to_csv('model_comparison.csv', index=False)
print("Comparison results saved to model_comparison.csv")


Loading datasets...

===== Implementing Logistic Regression from Scratch =====
Training logistic regression...
Iteration 0, Loss: 0.6907
Iteration 100, Loss: 0.5546
Iteration 200, Loss: 0.5097
Iteration 300, Loss: 0.4909
Iteration 400, Loss: 0.4818
Iteration 500, Loss: 0.4769
Iteration 600, Loss: 0.4739
Iteration 700, Loss: 0.4720
Iteration 800, Loss: 0.4706
Iteration 900, Loss: 0.4696
Training completed in 0.8267 seconds
Evaluating on test set...
Logistic Regression results saved in LogisticRegression directory

===== Implementing Decision Tree Classifier from Scratch =====
Training decision tree...
Training completed in 155.4038 seconds
Evaluating on test set...
Decision Tree results saved in DecisionTree directory

===== Implementing K-Nearest Neighbors Classifier from Scratch =====
Training KNN...
Training completed in 0.0001 seconds
Evaluating on test set...
KNN results saved in KNN directory

===== Comparing All Models =====

Model Comparison:
                 Model  Accuracy  Pr